In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
import gc
from collections import Counter

In [2]:
class Config:
    TRAIN_PATH = '../inputs/train.csv'
    TEST_PATH = '../inputs/test.csv'
    TRAIN_LABELS = '../inputs/train_labels.csv'
    SAMPLE_SUBMISSION = '../inputs/sample_submission.csv'

In [3]:
train_df = pd.read_csv(Config.TRAIN_PATH)
train_labels = pd.read_csv(Config.TRAIN_LABELS)

In [6]:
train_labels['question'] = train_labels['session_id'].str.split('q').str[-1].astype('int8')
train_labels['session'] = train_labels['session_id'].str.split('_').str[0]
train_labels.head()

,session_id,correct,question,session
0,20090312431273200_q1,1,1,20090312431273200
1,20090312433251036_q1,0,1,20090312433251036
2,20090314121766812_q1,1,1,20090314121766812
3,20090314363702160_q1,1,1,20090314363702160
4,20090314441803444_q1,1,1,20090314441803444


In [18]:
train_labels[(train_labels.question==2)&(train_labels.correct==1)]

,session_id,correct,question,session
11779,20090312431273200_q2,1,2,20090312431273200
11780,20090312433251036_q2,1,2,20090312433251036
11781,20090314121766812_q2,1,2,20090314121766812
11782,20090314363702160_q2,1,2,20090314363702160
11783,20090314441803444_q2,1,2,20090314441803444
...,...,...,...,...
23553,22100215342220508_q2,1,2,22100215342220508
23554,22100215460321130_q2,1,2,22100215460321130
23555,22100217104993650_q2,1,2,22100217104993650
23556,22100219442786200_q2,1,2,22100219442786200


In [19]:
train_labels[(train_labels.question==2)&(train_labels.correct==0)]

,session_id,correct,question,session
11788,20090317080721164_q2,0,2,20090317080721164
11856,20100110204440016_q2,0,2,20100110204440016
11917,20100115142089760_q2,0,2,20100115142089760
11922,20100115262106308_q2,0,2,20100115262106308
12066,20100308482129564_q2,0,2,20100308482129564
...,...,...,...,...
23175,22080409344323336_q2,0,2,22080409344323336
23290,22080610560827704_q2,0,2,22080610560827704
23330,22090108192456930_q2,0,2,22090108192456930
23446,22090413294967440_q2,0,2,22090413294967440


In [56]:
train_df[(train_df.session_id==20090312431273200)&(train_df.level_group=='0-4')]['text'].values

array(['undefined', 'Whatcha doing over there, Jo?',
       'Just talking to Teddy.', 'I gotta run to my meeting!',
       'Can I come, Gramps?',
       'Sure thing, Jo. Grab your notebook and come upstairs!',
       'See you later, Teddy.', "I get to go to Gramps's meeting!",
       'Now where did I put my notebook?', '\\u00f0\\u0178\\u02dc\\u00b4',
       nan, '\\u00f0\\u0178\\u02dc\\u00b4', nan,
       'I love these photos of me and Teddy!', nan, nan, nan, nan, nan,
       nan, nan, nan, nan, nan, nan, nan, 'Found it!', nan, nan,
       'Gramps is in trouble for losing papers?', "This can't be right!",
       'Gramps is a great historian!', nan, nan, nan, nan, nan,
       "Hmm. Button's still not working.", nan, nan, nan,
       "Let's get started. The Wisconsin Wonders exhibit opens tomorrow!",
       'Who wants to investigate the shirt artifact?',
       "Not Leopold here. He's been losing papers lately.", 'Hey!',
       "It's true, they do keep going missing lately.", 'See?',
   

In [57]:
train_df[(train_df.session_id==20090317080721164)&(train_df.level_group=='0-4')]['text'].values

array(['undefined', 'Whatcha doing over there, Jo?',
       'Just talking to Teddy.', 'I gotta run to my meeting!',
       'Can I come, Gramps?',
       'Sure thing, Jo. Grab your notebook and come upstairs!',
       'See you later, Teddy.', "I get to go to Gramps's meeting!",
       'Now where did I put my notebook?', '\\u00f0\\u0178\\u02dc\\u00b4',
       nan, nan, '\\u00f0\\u0178\\u02dc\\u00b4', nan, nan, nan, nan, nan,
       nan, nan, nan, nan, nan, nan,
       'Gramps is in trouble for losing papers?', "This can't be right!",
       'Gramps is the best historian ever!', nan, nan, 'Found it!', nan,
       nan, nan, nan, nan, nan,
       "Let's get started. The Wisconsin Wonders exhibit opens tomorrow!",
       'Who wants to investigate the shirt artifact?',
       "Not Leopold here. He's been losing papers lately.", 'Hey!',
       "It's true, they do keep going missing lately.", 'See?',
       'Besides, I already figured out the shirt.',
       "It's a women's basketball jersey!",

In [50]:
def find(sesssion_id):
    start = train_df[(train_df.session_id==sesssion_id)&(train_df.level_group=='0-4')&(train_df.text=="I'll be at the Capitol. Let me know if you find anything!")]['index'].values[0]
    end = train_df[(train_df.session_id==sesssion_id)&(train_df.level_group=='0-4')&(train_df.text=="The slip is from 1916 but the team didn't start until 1974!")]['index'].values[0]
    return start,end

In [54]:
start, end = find(20090312431273200)
train_df[(train_df.session_id==20090312431273200)&(start<=train_df['index'])&(train_df['index']<=end)]

,session_id,index,elapsed_time,event_name,name,level,page,room_coor_x,room_coor_y,screen_coor_x,screen_coor_y,hover_duration,text,fqid,room_fqid,text_fqid,fullscreen,hq,music,level_group
98,20090312431273200,98,100713,person_click,basic,2,NaN,400.098469,-176.662028,698.0,451.0,NaN,I'll be at the Capitol. Let me know if you fin...,gramps,tunic.historicalsociety.collection,tunic.historicalsociety.collection.gramps.found,NaN,NaN,NaN,0-4
99,20090312431273200,99,101042,navigate_click,undefined,3,NaN,400.098371,-175.222667,698.0,450.0,NaN,NaN,NaN,tunic.historicalsociety.collection,NaN,NaN,NaN,NaN,0-4
100,20090312431273200,100,101759,navigate_click,undefined,3,NaN,590.691801,-139.606888,806.0,422.0,NaN,NaN,NaN,tunic.historicalsociety.collection,NaN,NaN,NaN,NaN,0-4
101,20090312431273200,101,102324,navigate_click,undefined,3,NaN,638.630233,-138.925759,807.0,421.0,NaN,NaN,toentry,tunic.historicalsociety.collection,NaN,NaN,NaN,NaN,0-4
102,20090312431273200,102,103491,navigate_click,undefined,3,NaN,17.175558,-112.718163,750.0,391.0,NaN,NaN,NaN,tunic.historicalsociety.entry,NaN,NaN,NaN,NaN,0-4
103,20090312431273200,103,104309,navigate_click,undefined,3,NaN,-0.131890,-59.286580,706.0,357.0,NaN,NaN,NaN,tunic.historicalsociety.entry,NaN,NaN,NaN,NaN,0-4
104,20090312431273200,104,105058,navigate_click,undefined,3,NaN,112.618149,-60.992758,704.0,361.0,NaN,NaN,NaN,tunic.historicalsociety.entry,NaN,NaN,NaN,NaN,0-4
105,20090312431273200,105,105775,navigate_click,undefined,3,NaN,103.863973,77.120849,610.0,277.0,NaN,NaN,tostacks,tunic.historicalsociety.entry,NaN,NaN,NaN,NaN,0-4
106,20090312431273200,106,108090,navigate_click,undefined,3,NaN,55.000000,-59.434545,495.0,372.0,NaN,NaN,NaN,tunic.historicalsociety.stacks,NaN,NaN,NaN,NaN,0-4
107,20090312431273200,107,108924,navigate_click,undefined,3,NaN,-23.755212,162.467497,404.0,156.0,NaN,NaN,outtolunch,tunic.historicalsociety.stacks,NaN,NaN,NaN,NaN,0-4


In [58]:
start, end = find(20100115142089760)
train_df[(train_df.session_id==20100115142089760)&(start<=train_df['index'])&(train_df['index']<=end)]

,session_id,index,elapsed_time,event_name,name,level,page,room_coor_x,room_coor_y,screen_coor_x,screen_coor_y,hover_duration,text,fqid,room_fqid,text_fqid,fullscreen,hq,music,level_group
151098,20100115142089760,84,79083,person_click,basic,2,NaN,-159.825471,-28.404351,309.0,348.0,NaN,I'll be at the Capitol. Let me know if you fin...,gramps,tunic.historicalsociety.collection,tunic.historicalsociety.collection.gramps.found,NaN,NaN,NaN,0-4
151099,20100115142089760,85,79484,navigate_click,undefined,3,NaN,-159.825714,-28.404432,309.0,348.0,NaN,NaN,gramps,tunic.historicalsociety.collection,NaN,NaN,NaN,NaN,0-4
151100,20100115142089760,86,80549,person_click,basic,3,NaN,515.249849,-97.495406,778.0,396.0,NaN,Um... what did you want me to do again?,gramps,tunic.historicalsociety.collection,tunic.historicalsociety.collection.gramps.lost,NaN,NaN,NaN,0-4
151101,20100115142089760,87,81370,person_click,basic,3,NaN,618.886190,-109.010565,850.0,404.0,NaN,Head over to the Basketball Center.,gramps,tunic.historicalsociety.collection,tunic.historicalsociety.collection.gramps.lost,NaN,NaN,NaN,0-4
151102,20100115142089760,88,81749,person_click,basic,3,NaN,509.492245,-100.374204,774.0,398.0,NaN,Hopefully you can rustle up some clues!,gramps,tunic.historicalsociety.collection,tunic.historicalsociety.collection.gramps.lost,NaN,NaN,NaN,0-4
151103,20100115142089760,89,82648,navigate_click,undefined,3,NaN,509.492242,-100.374205,774.0,398.0,NaN,NaN,NaN,tunic.historicalsociety.collection,NaN,NaN,NaN,NaN,0-4
151104,20100115142089760,90,84840,navigate_click,undefined,3,NaN,676.616647,-107.980813,822.0,401.0,NaN,NaN,toentry,tunic.historicalsociety.collection,NaN,NaN,NaN,NaN,0-4
151105,20100115142089760,91,86660,navigate_click,undefined,3,NaN,15.547194,-109.461436,749.0,389.0,NaN,NaN,NaN,tunic.historicalsociety.entry,NaN,NaN,NaN,NaN,0-4
151106,20100115142089760,92,86812,navigate_click,undefined,3,NaN,250.006593,-97.741145,876.0,381.0,NaN,NaN,NaN,tunic.historicalsociety.entry,NaN,NaN,NaN,NaN,0-4
151107,20100115142089760,93,87312,navigate_click,undefined,3,NaN,310.063531,-97.267814,876.0,381.0,NaN,NaN,NaN,tunic.historicalsociety.entry,NaN,NaN,NaN,NaN,0-4


In [9]:
train_df.drop(columns=["fullscreen", "hq", "music"],inplace=True)

In [11]:
CATS = ['event_name', 'fqid', 'room_fqid', 'text']
NUMS = ['elapsed_time','page','room_coor_x', 'room_coor_y',
        'screen_coor_x', 'screen_coor_y', 'hover_duration']
EVENTS = ['navigate_click','person_click','cutscene_click','object_click',
          'map_hover','notification_click','map_click','observation_click',
          'checkpoint']
ROOMS = ['historicalsociety',
          'kohlcenter',
          'capitol_0',
          'humanecology',
          'drycleaner',
          'library',
          'capitol_1',
          'wildlife',
          'flaghouse',
          'capitol_2']
FQID = ['intro', 'gramps', 'teddy', 'photo', 'notebook',
       'retirement_letter', 'tobasement', 'janitor', 'toentry',
       'groupconvo', 'report', 'boss', 'wells', 'directory',
       'tocollection', 'cs', 'tunic', 'tunic.hub.slip', 'tostacks',
       'outtolunch', 'tocloset', 'tomap', 'tunic.historicalsociety',
       'tunic.kohlcenter', 'plaque', 'plaque.face.date', 'togrampa',
       'tunic.capitol_0', 'chap1_finale', 'chap1_finale_c',
       'tocloset_dirty', 'what_happened', 'trigger_scarf',
       'trigger_coffee', 'tunic.capitol_1', 'tofrontdesk', 'archivist',
       'magnify', 'tunic.humanecology', 'worker', 'businesscards',
       'businesscards.card_0.next', 'businesscards.card_1.next',
       'businesscards.card_bingo.next', 'businesscards.card_bingo.bingo',
       'tohallway', 'tunic.drycleaner', 'logbook', 'logbook.page.bingo',
       'tunic.library', 'tomicrofiche', 'reader', 'reader.paper0.next',
       'reader.paper1.next', 'reader.paper2.bingo', 'wellsbadge',
       'journals', 'journals.hub.topics', 'journals.pic_0.next',
       'journals.pic_1.next', 'journals.pic_2.bingo', 'chap2_finale_c',
       'ch3start', 'seescratches', 'tocage', 'glasses',
       'directory.closeup.archivist', 'key', 'unlockdoor',
       'confrontation', 'savedteddy', 'tocollectionflag',
       'groupconvo_flag', 'tunic.capitol_2', 'tunic.wildlife', 'coffee',
       'crane_ranger', 'remove_cup', 'expert', 'tracks',
       'tracks.hub.deer', 'tunic.flaghouse', 'flag_girl', 'colorbook',
       'reader_flag', 'reader_flag.paper0.next',
       'reader_flag.paper1.next', 'reader_flag.paper2.bingo',
       'archivist_glasses', 'journals_flag',
       'journals_flag.hub.topics_old', 'journals_flag.hub.topics',
       'journals_flag.pic_0.bingo', 'journals_flag.pic_0.next',
       'chap4_finale_c', 'block_tocollection', 'reader.paper2.next',
       'journals.pic_2.next', 'lockeddoor', 'reader.paper2.prev',
       'reader.paper0.prev', 'reader_flag.paper1.prev',
       'journals_flag.pic_0_old.next', 'journals_flag.pic_1_old.next',
       'door_block_clean', 'door_block_talk', 'block',
       'reader_flag.paper2.next', 'journals_flag.pic_1.bingo',
       'journals_flag.pic_1.next', 'journals_flag.pic_2.bingo',
       'journals_flag.pic_2.next', 'reader_flag.paper0.prev',
       'reader.paper1.prev', 'block_magnify',
       'journals_flag.pic_2_old.next', 'block_0', 'doorblock',
       'block_tomap1', 'block_tomap2', 'reader_flag.paper2.prev',
       'need_glasses', 'block_badge', 'block_nelson', 'block_badge_2',
       'block_1', 'fox']

In [12]:
def groupby_apply_elapsed_time(g):
    res = {}
    diff = np.diff(g['elapsed_time'].values)
    res['elapsed_time_diff_mean'] = np.mean(diff)
    res['elapsed_time_diff_std'] = np.std(diff)
    res['elapsed_time_positive'] = len(diff[diff>0])
    res['elapsed_time_negative'] = len(diff) - res['elapsed_time_positive']
    res['start-end'] = g['elapsed_time'].values[-1] - g['elapsed_time'].values[0]
    return pd.Series(res)

In [13]:
def feature_engineer(train):
    dfs = []
    tmp = train.groupby(['session_id','level_group']).apply(groupby_apply_elapsed_time)
    dfs.append(tmp)

    for c in EVENTS:
        train[c] = (train.event_name == c)
    train['main_room'] = train['room_fqid'].str.split('.').str[1]
    for c in ROOMS:
        train[c] = (train.main_room == c)
    for c in FQID:
        train[c] = (train.fqid==c)
    for c1 in ROOMS:
        for c2 in EVENTS:
            train[f'{c1}_{c2}'] = train[c1] & train[c2]
            tmp = train.groupby(['session_id','level_group'])[f'{c1}_{c2}'].sum()
            tmp.name = tmp.name + '_sum'
            dfs.append(tmp)
    for c in EVENTS +ROOMS+ ['elapsed_time']:
        tmp = train.groupby(['session_id','level_group'])[c].agg('sum')
        tmp.name = tmp.name + '_sum'
        dfs.append(tmp)
    for c in CATS+FQID:
        tmp = train.groupby(['session_id','level_group'])[c].agg('nunique')
        tmp.name = tmp.name + '_nunique'
        dfs.append(tmp)
    for c in NUMS + EVENTS + ROOMS:
        tmp = train.groupby(['session_id','level_group'])[c].agg('mean')
        tmp.name = tmp.name + '_mean'
        dfs.append(tmp)
    for c in NUMS+ EVENTS + ROOMS:
        tmp = train.groupby(['session_id','level_group'])[c].agg('std')
        tmp.name = tmp.name + '_std'
        dfs.append(tmp)
    for c in ['room_coor_x', 'room_coor_y',
        'screen_coor_x', 'screen_coor_y']:
        tmp = train.groupby(['session_id','level_group'])[c].agg('median')
        tmp.name = tmp.name + '_median'
        dfs.append(tmp)
    train.drop(columns=EVENTS+ROOMS,inplace=True)

    df = pd.concat(dfs,axis=1)
    df = df.fillna(-1)
    df = df.reset_index()
    df = df.set_index('session_id')
    return df

In [ ]:
%%time
df = feature_engineer(train_df)
print( df.shape )
df.head()

C:\Users\PC1\AppData\Local\Temp\ipykernel_2192\3158121175.py:12: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  train[c] = (train.fqid==c)
C:\Users\PC1\AppData\Local\Temp\ipykernel_2192\3158121175.py:12: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  train[c] = (train.fqid==c)
C:\Users\PC1\AppData\Local\Temp\ipykernel_2192\3158121175.py:12: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.